In [4]:
# loading libraries 

import matplotlib as plt
import pandas as pd
import os
import pycountry
import string
from unidecode import unidecode
from fuzzywuzzy import fuzz, process

path = os.getcwd()

In [5]:
# loading the data

ucdp = pd.read_csv(path + "/data/ucdp_grouped.csv")
gdelt = pd.read_csv(path + "/data/final_gdelt_bycountry.txt")

In [6]:
ucdp.head(10)

,month,year,adm_1,country,country_id,region,best,best_state,best_nonstate,best_onesided,deaths_civilians,number_of_sources,count_conflict_new_id,freq_conflict_new_id,count_dyad_new_id,freq_dyad_new_id,MonthYear
0,1,1989,Agusan del Sur province,Philippines,840,Asia,5,0.0,0.0,5.0,5,0.0,1,495,1,962,198901
1,1,1989,Alta Verapaz department,Guatemala,90,Americas,4,4.0,0.0,0.0,0,0.0,1,233,1,469,198901
2,1,1989,Ancash department,Peru,135,Americas,4,4.0,0.0,0.0,0,0.0,1,292,1,631,198901
3,1,1989,Antioquia department,Colombia,100,Americas,0,0.0,0.0,0.0,0,0.0,1,605,1,1072,198901
4,1,1989,Apurímac department,Peru,135,Americas,17,17.0,0.0,0.0,0,0.0,1,292,1,631,198901
5,1,1989,Arequipa department,Peru,135,Americas,2,2.0,0.0,0.0,0,1.0,1,292,1,631,198901
6,1,1989,Assam state,India,750,Asia,35,35.0,0.0,0.0,0,2.0,1,421,1,884,198901
7,1,1989,Ayacucho department,Peru,135,Americas,25,22.0,0.0,3.0,3,0.0,2,292,2,631,198901
8,1,1989,Azerbaijan SSR,Russia (Soviet Union),365,Europe,42,0.0,42.0,0.0,0,0.0,1,4809,1,5419,198901
9,1,1989,Baghlan province,Afghanistan,700,Asia,670,670.0,0.0,0.0,0,0.0,1,333,1,724,198901


In [4]:
gdelt.head(10)

,ActionGeo_ADM1Code,isocode,year,month,count_events_1,count_events_2,count_events_3,count_events_4,count_events_5,count_events_6,...,count_events_20_opp,Actor1Code_count,Actor1CountryCode_count,Actor1KnownGroupCode_count,Actor1Type1Code_count,Actor2Code_count,Actor2CountryCode_count,Actor2KnownGroupCode_count,Actor2Type1Code_count,AvgGoldsteinScale
0,AA,ABW,1920,1,2.0,0.0,0.0,2.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,AA,ABW,1981,10,2.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,AA,ABW,1981,12,0.0,0.0,0.0,22.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,AA,ABW,1983,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,AA,ABW,1983,2,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
5,AA,ABW,1983,3,2.0,6.0,6.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
6,AA,ABW,1983,4,2.0,0.0,0.0,8.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
7,AA,ABW,1983,5,2.0,0.0,4.0,0.0,2.0,0.0,...,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,3.4
8,AA,ABW,1983,12,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
9,AA,ABW,1984,1,0.0,0.0,0.0,4.0,0.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [6]:
# Dropping columns before 1989

ucdp.drop(ucdp.loc[ucdp['year']<1989].index, inplace=True)
gdelt.drop(gdelt.loc[gdelt['year']<1989].index, inplace=True)

In [7]:
# Verifying Russia in UCDP

ucdp[['MonthYear','adm_1','country']].loc[ucdp['country']== "Russia (Soviet Union)"]

,MonthYear,adm_1,country
1164,200001,Chechnya republic,Russia (Soviet Union)
1281,200101,Chechnya republic,Russia (Soviet Union)
1409,200201,Chechnya republic,Russia (Soviet Union)
1415,200201,Dagestan republic,Russia (Soviet Union)
1638,200401,Chechnya republic,Russia (Soviet Union)
...,...,...,...
52393,202012,Karachay-Cherkessia republic,Russia (Soviet Union)
52686,202212,Belgorod oblast,Russia (Soviet Union)
52690,202212,Bryansk oblast,Russia (Soviet Union)
52819,202212,Ryazan oblast,Russia (Soviet Union)


In [8]:
# Verifying Russia in GDELT

gdelt[['year','ActionGeo_ADM1Code','isocode']].loc[gdelt['isocode']== "RUS"]

,year,ActionGeo_ADM1Code,isocode
849115,2000,RS,RUS
849116,2000,RS,RUS
849117,2000,RS,RUS
849118,2000,RS,RUS
849119,2000,RS,RUS
...,...,...,...
888532,2023,RSJA,RUS
888533,2023,RSJA,RUS
888534,2023,RSJA,RUS
888535,2023,RSJA,RUS


In [9]:
# Verifying Armenia in UCDP

ucdp[['MonthYear','adm_1','country']].loc[ucdp['country']== "Armenia"]

,MonthYear,adm_1,country
3139,201501,Shirak province,Armenia
3151,201501,Tavush province,Armenia
6906,201202,Tavush province,Armenia
7318,201502,Tavush province,Armenia
10474,200503,Tavush province,Armenia
11397,201203,Tavush province,Armenia
13234,202303,Ararat province,Armenia
16112,201204,Tavush province,Armenia
19936,200605,Tavush province,Armenia
22278,202105,Geghark’unik’ province,Armenia


In [10]:
# Verifying Armenia in GDELT

gdelt[['year','ActionGeo_ADM1Code','isocode']].loc[gdelt['isocode']== "ARM"]

,year,ActionGeo_ADM1Code,isocode
52766,2000,AM,ARM
52767,2000,AM,ARM
52768,2000,AM,ARM
52769,2000,AM,ARM
52770,2000,AM,ARM
...,...,...,...
57046,2023,AM11,ARM
57047,2023,AM11,ARM
57048,2023,AM11,ARM
57049,2023,AM11,ARM


## Exploring UCDP's Admin 1 data

In [11]:
# standardizing the names

ucdp['adm_new'] = ucdp['adm_1'].str.rsplit(n=1).str[0]
ucdp['adm_new'] = ucdp['adm_new'].apply(lambda x: unidecode(x).translate(str.maketrans("", "", string.punctuation)).strip().lower())

ucdp.head(10)

,month,year,adm_1,country,country_id,region,best,best_state,best_nonstate,best_onesided,deaths_civilians,number_of_sources,count_conflict_new_id,freq_conflict_new_id,count_dyad_new_id,freq_dyad_new_id,MonthYear,adm_new
1139,1,2000,Aceh province,Indonesia,850,Asia,72,68.0,0.0,4.0,4,0.00,2,366,2,794,200001,aceh
1140,1,2000,Ain Defla province,Algeria,615,Africa,12,0.0,0.0,12.0,12,0.00,1,572,1,1039,200001,ain defla
1141,1,2000,Andhra Pradesh state,India,750,Asia,5,5.0,0.0,0.0,0,0.00,1,227,1,451,200001,andhra pradesh
1142,1,2000,Antioquia department,Colombia,100,Americas,72,8.0,0.0,64.0,64,0.72,4,610,5,1077,200001,antioquia
1143,1,2000,Anzoátegui state,Venezuela,101,Americas,0,0.0,0.0,0.0,0,1.00,1,447,1,914,200001,anzoategui
1144,1,2000,Aragua state,Venezuela,101,Americas,1,0.0,0.0,1.0,1,1.00,1,447,1,914,200001,aragua
1145,1,2000,Aračinovo municipality,"Macedonia, FYR",343,Europe,3,3.0,0.0,0.0,0,0.00,1,417,1,877,200001,aracinovo
1146,1,2000,Assam state,India,750,Asia,21,18.0,0.0,3.0,4,0.00,3,365,3,793,200001,assam
1147,1,2000,Azad Jammu & Kashmir,Pakistan,770,Asia,2,2.0,0.0,0.0,2,1.00,1,218,1,422,200001,azad jammu
1148,1,2000,Bagmati zone,Nepal,790,Asia,6,4.0,0.0,2.0,2,0.00,3,269,3,562,200001,bagmati


In [12]:
# creating fuzzy function

def fuzzy_match(name, candidates):
    matches = process.extractOne(name, candidates)
    if matches[1] >= 50:  # Adjust the threshold as per your needs
        return matches[0]
    else:
        return None


In [13]:
gdelt_data = {
    'adm_new': ['beirut', 'cairo', 'paris', 'london'],
    'data': [1, 2, 3, 4]
}

ucdp_data = {
    'adm_new': ['beyrouth', 'kairo', 'pari', 'londn', 'bierooth'],
    'iso_code': ['LBN', 'EGY', 'FRA', 'GBR', 'LBN'],
    'population': [2000000, 10000000, 2200000, 9000000, 2000000]
}

gdelt_data = pd.DataFrame(gdelt_data)
ucdp_data = pd.DataFrame(ucdp_data)

In [14]:
gdelt_data['matched_name'] = gdelt_data['adm_new'].apply(lambda x: fuzzy_match(x, ucdp_data['adm_new'].tolist()))
merged_df = gdelt_data.merge(ucdp_data, left_on='matched_name', right_on='adm_new', how='left')

merged_df.head(10)

,adm_new_x,data,matched_name,adm_new_y,iso_code,population
0,beirut,1,beyrouth,beyrouth,LBN,2000000
1,cairo,2,kairo,kairo,EGY,10000000
2,paris,3,pari,pari,FRA,2200000
3,london,4,londn,londn,GBR,9000000


In [ ]:
from fuzzywuzzy import fuzz

def fuzzy_match(string, candidates):
    best_ratio = -1
    best_candidate = None

    for candidate in candidates:
        ratio = fuzz.ratio(string, candidate)
        if ratio > best_ratio:
            best_ratio = ratio
            best_candidate = candidate

    return best_candidate

def get_admin_iso(df):
    subdivisions = pycountry.subdivisions

    for index, row in df.iterrows():
        subdivision_name = row['adm_new']
        subdivision_name = unidecode(subdivision_name).translate(str.maketrans("", "", string.punctuation)).strip().lower()

        best_match = None
        best_ratio = -1

        for subdivision in subdivisions:
            candidate_name = str(subdivision.name)
            candidate_name = unidecode(candidate_name).translate(str.maketrans("", "", string.punctuation)).strip().lower()
            ratio = fuzz.ratio(subdivision_name, candidate_name)

            if ratio > best_ratio:
                best_ratio = ratio
                best_match = subdivision

        if best_match:
            df.at[index, 'iso_code'] = best_match.code

    return df

get_admin_iso(ucdp)
ucdp.head(10)

In [ ]:
# retrieving subdivision isocode from pycountry
def get_admin_iso(df):
    subdivisions = pycountry.subdivisions

    for subdivision in subdivisions:
        subdivision_name = str(subdivision.name)
        subdivision_name = unidecode(subdivision_name).translate(str.maketrans("", "", string.punctuation)).strip().lower()
        
        matching_rows = df[df['adm_new'] == subdivision_name]

        if not matching_rows.empty:
            df.loc[matching_rows.index, 'iso_code'] = subdivision.code

    return df

get_admin_iso(ucdp)
ucdp.head(10)

In [ ]:
# exploring unmatched iso codes

print(ucdp['iso_code'].isna().sum()/len(ucdp['iso_code']))
nan_adm_new = ucdp.loc[ucdp['iso_code'].isna(), 'adm_new'].unique().tolist()
len(nan_adm_new)

In [ ]:
# examples of cases where fuzzy matching must be done

In [ ]:
print(pycountry.subdivisions.get(code="PE-APU"))

ucdp[['country','adm_1', 'adm_new','iso_code']].loc[ucdp['adm_new']=="apurimac"]

In [ ]:
print(pycountry.subdivisions.get(code="ES-MD"))

ucdp[['country','adm_1', 'adm_new','iso_code']].loc[ucdp['adm_new']=="community of"]

In [ ]:
print(pycountry.subdivisions.get(code="ES-VC"))

ucdp[['country','adm_1', 'adm_new','iso_code']].loc[ucdp['adm_new']=="valenciana autonomous"]

In [ ]:
print(pycountry.subdivisions.get(code="TG-M"))

ucdp[['country','adm_1', 'adm_new','iso_code']].loc[ucdp['adm_new']=="maritime"]

In [ ]:
# examples of cases where matching must be at the country level

ucdp[['country','adm_1', 'adm_new','iso_code']].loc[ucdp['adm_new']=="armenian"]

## Exploring GDELT's Admin 1 data

In [ ]:
# retrieving subdivision names from FIPS codes

print("\n****************\n\nGetting names from FIPS code...\n")
fips = pd.read_csv(path +'/data/fipsadm.csv') # codes' list

fips['adm_new'] = fips['region_name'].apply(lambda x: unidecode(x).translate(str.maketrans("", "", string.punctuation)).strip().lower())

fips.head(10)

In [ ]:
# merging them:

gdelt['ActionGeo_ADM1Code'] = gdelt['ActionGeo_ADM1Code'].apply(lambda x: x + '00' if len(x) == 2 else x)
gdelt = pd.merge(gdelt,fips,how='left', left_on = 'ActionGeo_ADM1Code', right_on = 'region_code')

# removing useless variables:
gdelt = gdelt.drop(columns=['region_code', 'region_name'])

gdelt.head(10)

In [ ]:
# exploring unmatched names

print(gdelt['adm_new'].isna().sum()/len(gdelt['adm_new']))

nans_gdelt = gdelt.loc[gdelt['adm_new'].isna(), 'ActionGeo_ADM1Code'].unique().tolist()
len(nans_gdelt)

In [ ]:
# examples of cases where code does not match

In [ ]:
print(fips[['region_code','region_name','adm_new']].loc[fips['region_code']=='AO10'])
print(fips.loc[fips['adm_new']=='luanda'])

## AO10 no longer exists, but used to refer to Luanda, which is now AO20

print(gdelt[['ActionGeo_ADM1Code','isocode','adm_new']].loc[gdelt['ActionGeo_ADM1Code']=='AO10'])
print(gdelt[['ActionGeo_ADM1Code','isocode','adm_new']].loc[gdelt['ActionGeo_ADM1Code']=='AO20'])

In [ ]:
print(fips[['region_code','region_name','adm_new']].loc[fips['region_code']=='BE13'])
print(fips[['region_code','region_name','adm_new']].loc[fips['region_code']=='BE08'])
print(fips[['region_code','region_name','adm_new']].loc[fips['region_code']=='BE09'])
print(fips[['region_code','region_name','adm_new']].loc[fips['region_code']=='BE12'])

## BE13 no longer exists but used to refer to the Flanders region, which is not subdivided in GDELT but is in FIPS

print(gdelt[['ActionGeo_ADM1Code','isocode','adm_new']].loc[gdelt['ActionGeo_ADM1Code']=='BE13'])
print(gdelt[['ActionGeo_ADM1Code','isocode','adm_new']].loc[gdelt['ActionGeo_ADM1Code']=='BE08'])
print(gdelt[['ActionGeo_ADM1Code','isocode','adm_new']].loc[gdelt['ActionGeo_ADM1Code']=='BE09'])
print(gdelt[['ActionGeo_ADM1Code','isocode','adm_new']].loc[gdelt['ActionGeo_ADM1Code']=='BE12'])

In [ ]:
print(fips[['region_code','region_name','adm_new']].loc[fips['region_code']=='BE14'])
print(fips[['region_code','region_name','adm_new']].loc[fips['region_code']=='BE03'])
print(fips[['region_code','region_name','adm_new']].loc[fips['region_code']=='BE04'])
print(fips[['region_code','region_name','adm_new']].loc[fips['region_code']=='BE06'])
print(fips[['region_code','region_name','adm_new']].loc[fips['region_code']=='BE07'])
print(fips[['region_code','region_name','adm_new']].loc[fips['region_code']=='BE10'])

## BE14 no longer exists but used to refer to Wallonia (now subdivided in FIPS but not in GDELT)

print(gdelt[['ActionGeo_ADM1Code','isocode','adm_new']].loc[gdelt['ActionGeo_ADM1Code']=='BE14'])
print(gdelt[['ActionGeo_ADM1Code','isocode','adm_new']].loc[gdelt['ActionGeo_ADM1Code']=='BE03'])
print(gdelt[['ActionGeo_ADM1Code','isocode','adm_new']].loc[gdelt['ActionGeo_ADM1Code']=='BE04'])
print(gdelt[['ActionGeo_ADM1Code','isocode','adm_new']].loc[gdelt['ActionGeo_ADM1Code']=='BE06'])
print(gdelt[['ActionGeo_ADM1Code','isocode','adm_new']].loc[gdelt['ActionGeo_ADM1Code']=='BE07'])
print(gdelt[['ActionGeo_ADM1Code','isocode','adm_new']].loc[gdelt['ActionGeo_ADM1Code']=='BE10'])

In [ ]:
# in ucdp, the division is the same as in gdelt though, which is good

ucdp[['country','adm_1', 'adm_new','iso_code']].loc[ucdp['country']=="Belgium"]

In [ ]:
# examples of cases where code is missing

In [ ]:
gdelt[['ActionGeo_ADM1Code','isocode','adm_new']].loc[gdelt['ActionGeo_ADM1Code']=='LGA1']

#print(fips[['region_code','region_name','adm_new']].loc[fips['adm_new']=='latvia'])

# nothing for subdivisions of Latvia whatsoever

In [ ]:
gdelt[['ActionGeo_ADM1Code','isocode','adm_new']].loc[gdelt['ActionGeo_ADM1Code']=='LGA2']

In [ ]:
ucdp[['country','adm_1', 'adm_new','iso_code']].loc[ucdp['country']=="Latvia"]

# no UCDP entries for Latvia either

## Merging UCDP and GDELT at the Admin 1 level

In [ ]:
gdelt['matched_name'] = gdelt['adm_new'].apply(lambda x: fuzzy_match(x, ucdp['adm_new'].tolist()))
merged_df = gdelt.merge(ucdp, left_on='matched_name', right_on='adm_new', how='left')

merged_df.head(10)

In [ ]:
merged_df = gdelt.merge(ucdp, left_on='matched_name', right_on='adm_new', how='left')


In [ ]:
unmatched_rows

In [ ]:
merged_df = gdelt.merge(ucdp, on='adm_new', how='outer', indicator=True)

In [ ]:
gdelt

In [ ]:
len(merged_df.loc[merged_df['_merge'] == 'left_only', 'adm_new'].unique())

In [ ]:
len(merged_df.loc[merged_df['_merge'] == 'right_only', 'adm_new'].unique())

In [ ]:
merged_df.loc[merged_df['_merge'] == 'right_only', 'adm_new'].unique()

In [ ]:
# regions where the names have different spelling

In [ ]:
ucdp[['country','adm_1', 'adm_new','iso_code']].loc[ucdp['adm_new']=="beirut"]

In [ ]:
gdelt[['ActionGeo_ADM1Code','isocode','adm_new']].loc[gdelt['adm_new']=='beyrouth']

##  Issues to deal with:

* UCDP:
    * Soviet Union is a country until 1991 and each Republic is an Admin 1 region. After 1991, each Republic is its own country with smaller Admin 1 regions.
    * 22% of the data is missing an ISO code (approx. 485 ISO codes) from pycountry.
    
* GDELT:
    * 11% of the data is missing a name tied to the FIPS code (approx. 699 names) from the FIPS.csv file

* Merging:
    * How do we merge country-level GDELT information with admin-level deaths?

In [ ]:
isocodes_to_drop = ['ABW', 'AIA', 'AND', 'ASM', 'BVT', 'COK', 'CXR', 'CYM', 'FLK', 'FRO', 'GGY', 'GLP', 'GUF', 'HMD', 'IMN', 'IOT', 'KIR', 'KNA', 'LCA', 'LIE', 'MTQ', 'MYT', 'NFK', 'NIU', 'NRU', 'PCN', 'PLW', 'PYF', 'REU', 'SHN', 'SJM', 'SMR', 'SPM', 'TCA', 'TKL', 'TUV', 'VCT', 'VGB', 'WLF']
gdelt_drop = gdelt[~gdelt['isocode'].isin(isocodes_to_drop)]

print(gdelt_drop['adm_new'].isna().sum()/len(gdelt_drop['adm_new']))

nans_gdelt = gdelt_drop.loc[gdelt_drop['adm_new'].isna(), 'ActionGeo_ADM1Code'].unique().tolist()
len(nans_gdelt)

In [ ]:
gdelt_drop['adm_new'].isna()